data location
https://www.kaggle.com/c/two-sigma-connect-rental-listing-inquiries/data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_train = pd.read_json('train.json')
data_test = pd.read_json('test.json')

In [3]:
data = data_train.append(data_test)

In [4]:
data = data.reindex(range(data.shape[0]))

In [5]:
import pickle

In [6]:
bedrooms_sim = pickle.load(open('bedroom_sim_mat.pickle','rb')) 
bathrooms_sim = pickle.load(open('bathroom_sim_mat.pickle','rb')) 
living_rooms_sim = pickle.load(open('living_room_sim_mat.pickle','rb')) 
kitchens_sim = pickle.load(open('kitchen_sim_mat.pickle','rb')) 

In [7]:
import os

In [8]:
house_id = os.listdir('house_image')

In [9]:
mask = data['listing_id'].apply(lambda x : 1 if str(x) in house_id else 0)

In [10]:
house_data = data.loc[mask == 1]

In [11]:
house_data = house_data.sort_values(by = ['listing_id'])

In [12]:
house_data = house_data.set_index('listing_id')

In [21]:
house_data.groupby(['bedrooms'])['bedrooms'].count()

bedrooms
1     1
2    87
3    31
4     6
Name: bedrooms, dtype: int64

In [22]:
description = pd.read_csv('description.txt', sep = '\n', header=None)

In [23]:
a = description.values.tolist()

In [24]:
b = [str(i).split(';')[1] for i in a]

In [ ]:
house_data['price'].hist(bins = 20)

In [28]:
house_data.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,longitude,manager_id,photos,price,street_address
listing_id,,,,,,,,,,,,,,
6812077,1.0,2,0,2016-04-02 01:14:33,Recent renovations including stainless steel k...,West 135th Street,[Pre-War],NaN,40.8160,-73.9445,e2d2576d07efdc1ed08cca7c43368f37,[https://photos.renthop.com/2/6812077_9d0749c6...,2100,213 West 135th Street
6812215,1.0,2,fcac3901a3fcd6833c853a0f8d5a7f9a,2016-04-02 01:20:09,Bright and Sunny 1300 Sqft Semi-Railroad 2 bed...,Noble Street,[No Fee],NaN,40.7284,-73.9560,3f949681e57d9432679999da9623db9f,[https://photos.renthop.com/2/6812215_36ee3519...,3800,116 Noble Street
6812282,3.0,3,0,2016-04-02 01:22:23,LOFT LIVING IN FLATIRON.Top notch finishes abo...,Fifth Avenue,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7382,-73.9918,55cbaa6667aed0eb6a7adb43d2f043b1,[https://photos.renthop.com/2/6812282_a01d5bb1...,13995,103 Fifth Avenue
6812939,1.0,2,ea9045106c4e1fe52853b6af941f1c69,2016-04-02 02:13:57,Call Marina Chukhriy for Appointments <BR> 997...,West 43rd Street,"[Balcony, Doorman, Elevator, Outdoor Space, Do...",NaN,40.7612,-73.9978,c23e16813063cb363bdd2904bd20d2d0,[https://photos.renthop.com/2/6812939_b80b374f...,5425,560 West 43rd Street
6812945,1.0,4,248b8377f9a18de9b86d645a86c90c9f,2016-04-02 02:14:08,Great for shares! Huge 4BR apartment on the ga...,3rd St,"[garden, Dishwasher, Private outdoor space, Pr...",NaN,40.6719,-73.9820,9e20f9acdbe2e965cea5380c659f01a9,[https://photos.renthop.com/2/6812945_e85338bc...,4000,410 3rd St


In [45]:
def build_house_database(entry, i):
    db = {}
    path = 'https://s3.amazonaws.com/room.pic/'
    db['index'] = str(i)
    db['listing_id'] = str(entry.name)
    if entry.bedrooms < 2:
        bedrooms = 2
    else:
        bedrooms = int(entry.bedrooms)
    db['size'] = "{}B{}B".format(bedrooms, int(entry.bathrooms))
    db['price'] = int(entry.price)
    db['description'] = b[i] # just strings
    db['features'] = entry.features # a list of features
    db['address'] = str(entry.street_address)
    db['bedrooms_sim'] = bedrooms_sim[i].tolist()
    db['bathrooms_sim'] = bathrooms_sim[i].tolist()
    db['kitchens_sim'] = kitchens_sim[i].tolist()
    db['living_rooms_sim'] = living_rooms_sim[i].tolist()
    db['bedroom_pic'] = path + str(entry.name) + '/bedroom.jpg' 
    db['bathroom_pic'] = path + str(entry.name) + '/bathroom.jpg' 
    db['kitchen_pic'] = path + str(entry.name) + '/kitchen.jpg' 
    db['living_room_pic'] = path + str(entry.name) + '/living_room.jpg'
    db['liked_by'] = []
    return db

In [46]:
house_db = []
for i in range(125):
    db = build_house_database(house_data.iloc[i],i)
    house_db.append(db)

In [47]:
with open('house_db.pickle','wb') as f:
    pickle.dump(house_db,f)

In [48]:
house_db[0]

{'address': '213 West 135th Street',
 'bathroom_pic': 'https://s3.amazonaws.com/room.pic/6812077/bathroom.jpg',
 'bathrooms_sim': [1.0,
  0.2156202495098114,
  0.16876472532749176,
  0.16536225378513336,
  0.21746399998664856,
  0.21833625435829163,
  0.20950670540332794,
  0.17587938904762268,
  0.17063255608081818,
  0.1289019137620926,
  0.19676122069358826,
  0.10729853808879852,
  0.23934459686279297,
  0.2819649875164032,
  0.2876381278038025,
  0.19543206691741943,
  0.18718180060386658,
  0.19219502806663513,
  0.1519227921962738,
  0.17777743935585022,
  0.28984785079956055,
  0.23113298416137695,
  0.18074272572994232,
  0.1349036991596222,
  0.26818302273750305,
  0.17561599612236023,
  0.19796815514564514,
  0.15209439396858215,
  0.2718314826488495,
  0.17907072603702545,
  0.13314242660999298,
  0.1673867553472519,
  0.1418229341506958,
  0.10561995208263397,
  0.11303120851516724,
  0.11759821325540543,
  0.23551659286022186,
  0.2048158198595047,
  0.22331923246383667,
